In [1]:
!pip install pandarallel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import polars as pl
import pandas as pd
import numpy as np
from joblib import Parallel,delayed
import time
from tqdm.notebook import tqdm
tqdm.pandas()
from pandarallel import pandarallel

# PolarsとPandas

In [3]:
data = {"col1": np.arange(100000000), "col2": ["a"]*30000000+["b"]*30000000+["c"]*40000000}
pd_df = pd.DataFrame(data, columns=["col1", "col2"])
pl_df = pl.DataFrame(data, schema=["col1","col2"])


In [14]:
%%time
pd_df.groupby("col2").sum()

CPU times: user 5.75 s, sys: 2.6 s, total: 8.35 s
Wall time: 8.37 s


,col1
col2,
a,449999985000000
b,1349999985000000
c,3199999980000000


In [15]:
%%time
pl_df.groupby("col2").sum()

CPU times: user 5.84 s, sys: 466 ms, total: 6.3 s
Wall time: 3.31 s


col2,col1
str,i64
"""c""",3199999980000000
"""b""",1349999985000000
"""a""",449999985000000


# 並列化関係


In [6]:
np.random.seed(1)
data2 = {"col1": np.arange(100000), "col2": np.random.randint(0,1000,100000)}
pd_df2 = pd.DataFrame(data2, columns=["col1", "col2"])


In [7]:
def parallel_func(x):
    time.sleep(0.01)
    return x[["col1"]].sum()

通常のapply

In [8]:
%%time
par1 = pd_df2.groupby("col2").progress_apply(parallel_func)

  0%|          | 0/1000 [00:00<?, ?it/s]

CPU times: user 2.7 s, sys: 91.1 ms, total: 2.79 s
Wall time: 12.8 s


pandarallel

In [9]:
%%time
# 初期化
pandarallel.initialize(nb_workers=2,progress_bar=True)

par2 = pd_df2.groupby("col2").parallel_apply(parallel_func)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


CPU times: user 1.01 s, sys: 283 ms, total: 1.3 s
Wall time: 8.49 s


joblib

In [10]:
def parallel_func2(col2,x):
    time.sleep(0.01)
    return [col2,x["col1"].sum()]

In [11]:
%%time
lists = Parallel(n_jobs=-1,verbose=1)([delayed(parallel_func2)(col2,gdf) for col2,gdf in pd_df2.groupby("col2")])
par3 = pd.DataFrame(lists,columns=["col2","col1"])

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 360 tasks      | elapsed:    3.3s


CPU times: user 310 ms, sys: 298 ms, total: 608 ms
Wall time: 6.89 s


[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:    6.8s finished
